##### Авторские права 2018 Авторы TensorFlow.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Использование формата SavedModel

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/saved_model"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Посмотреть на TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Запустить в Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Посмотреть исходный код на GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/saved_model.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Скачать блокнот</a></td>
</table>

SavedModel содержит полную программу TensorFlow, включая веса и вычисления. Он не требует наличия оригинальной модели здания код для запуска, что делает его полезным для совместного использования или развертывания (с [TFLite](https://tensorflow.org/lite) , [TensorFlow.js](https://js.tensorflow.org/) , [TensorFlow сервировки](https://www.tensorflow.org/tfx/serving/tutorials/Serving_REST_simple) или [TensorFlow Hub](https://tensorflow.org/hub) ).

В этом документе подробно описывается, как использовать низкоуровневый api `tf.saved_model`

- Если вы используете `tf.keras.Model` метод `keras.Model.save(output_path)` может быть всем, что вам нужно: см. [Сохранение и сериализация Keras](keras/save_and_serialize.ipynb)

- Если вы просто хотите сохранить / загрузить веса во время тренировки, см. [Руководство по контрольным точкам тренировки](./checkpoint.ipynb) .


## Создание SavedModel из Keras

Для быстрого введения в этом разделе экспортируется предварительно обученная модель Keras и обслуживается с ее помощью запросы классификации изображений. Остальная часть руководства заполнит детали и обсудит другие способы создания SavedModels.

In [0]:
import os
import tempfile

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

tmpdir = tempfile.mkdtemp()

In [0]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [0]:
file = tf.keras.utils.get_file(
    "grace_hopper.jpg",
    "https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg")
img = tf.keras.preprocessing.image.load_img(file, target_size=[224, 224])
plt.imshow(img)
plt.axis('off')
x = tf.keras.preprocessing.image.img_to_array(img)
x = tf.keras.applications.mobilenet.preprocess_input(
    x[tf.newaxis,...])

Мы будем использовать изображение Грейс Хоппер в качестве рабочего примера и предварительно обученную модель классификации изображений Keras, поскольку она проста в использовании. Также работают нестандартные модели, о которых мы подробнее поговорим позже.

In [0]:
labels_path = tf.keras.utils.get_file(
    'ImageNetLabels.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [0]:
pretrained_model = tf.keras.applications.MobileNet()
result_before_save = pretrained_model(x)

decoded = imagenet_labels[np.argsort(result_before_save)[0,::-1][:5]+1]

print("Result before saving:\n", decoded)

Главный прогноз для этого изображения - «военная форма».

In [0]:
mobilenet_save_path = os.path.join(tmpdir, "mobilenet/1/")
tf.saved_model.save(pretrained_model, mobilenet_save_path)

Путь сохранения следует соглашению, используемому службой TensorFlow, где последний компонент пути ( `1/` ) является номером версии для вашей модели - это позволяет таким инструментам, как Tensorflow Serving, определять относительную свежесть.

Мы можем загрузить SavedModel обратно в Python с помощью `tf.saved_model.load` и посмотреть, как классифицируется изображение адмирала Хоппера.

In [0]:
loaded = tf.saved_model.load(mobilenet_save_path)
print(list(loaded.signatures.keys()))  # ["serving_default"]

Импортированные подписи всегда возвращают словари. Чтобы настроить имена подписей и ключи выходного словаря, см. [Указание подписей во время экспорта](#specifying_signatures_during_export) .

In [0]:
infer = loaded.signatures["serving_default"]
print(infer.structured_outputs)

Выполнение вывода из SavedModel дает тот же результат, что и исходная модель.

In [0]:
labeling = infer(tf.constant(x))[pretrained_model.output_names[0]]

decoded = imagenet_labels[np.argsort(labeling)[0,::-1][:5]+1]

print("Result after saving and loading:\n", decoded)

## Запуск SavedModel в TensorFlow Serving

SavedModels можно использовать из Python (подробнее об этом ниже), но в производственных средах обычно используется выделенная служба для вывода без запуска кода Python. Это легко настроить из SavedModel с помощью TensorFlow Serving.

См. `tensorflow_model_server` [Serving REST](https://www.tensorflow.org/tfx/tutorials/serving/rest_simple) для получения дополнительных сведений об обслуживании, включая инструкции по установке tensorflow_model_server в записной книжке или на вашем локальном компьютере. В качестве быстрого наброска для обслуживания `mobilenet` просто укажите сервер модели в каталоге SavedModel:

```bash
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=mobilenet \
  --model_base_path="/tmp/mobilenet" >server.log 2>&1
```

Тогда отправьте заявку.

```python
!pip install requests
import json
import numpy
import requests
data = json.dumps({"signature_name": "serving_default",
                   "instances": x.tolist()})
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/mobilenet:predict',
                              data=data, headers=headers)
predictions = numpy.array(json.loads(json_response.text)["predictions"])
```

Полученные `predictions` идентичны результатам Python.

## Формат SavedModel на диске

SavedModel - это каталог, содержащий сериализованные подписи и состояние, необходимое для их запуска, включая значения переменных и словари.


In [0]:
!ls {mobilenet_save_path}

В `saved_model.pb` хранится фактическая программа или модель TensorFlow и набор именованных сигнатур, каждая из которых идентифицирует функцию, которая принимает тензорные входные данные и производит тензорные выходные данные.

SavedModels может содержать несколько вариантов модели (несколько `v1.MetaGraphDefs` , идентифицируемых с `--tag_set` флага `saved_model_cli` для saved_model_cli), но это бывает редко. API, которые создают несколько вариантов модели, включают [`tf.Estimator.experimental_export_all_saved_models`](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator#experimental_export_all_saved_models) и `tf.saved_model.Builder`

In [0]:
!saved_model_cli show --dir {mobilenet_save_path} --tag_set serve

`variables` содержит стандартную контрольную точку обучения (см. [Руководство по контрольным точкам обучения](./checkpoint.ipynb) ).

In [0]:
!ls {mobilenet_save_path}/variables

`assets` содержит файлы, используемые графиком TensorFlow, например текстовые файлы, используемые для инициализации словарных таблиц. В этом примере он не используется.

SavedModels может иметь `assets.extra` для любых файлов, не используемых графом TensorFlow, например, информацию для потребителей о том, что делать с SavedModel. Сам TensorFlow не использует этот каталог.

## Сохранение нестандартной модели

`tf.saved_model.save` поддерживает сохранение `tf.Module` и его подклассов, таких как `tf.keras.Layer` и `tf.keras.Model` .

Давайте посмотрим на пример сохранения и восстановления `tf.Module` .


In [0]:
class CustomModule(tf.Module):

  def __init__(self):
    super(CustomModule, self).__init__()
    self.v = tf.Variable(1.)

  @tf.function
  def __call__(self, x):
    print('Tracing with', x)
    return x * self.v

  @tf.function(input_signature=[tf.TensorSpec([], tf.float32)])
  def mutate(self, new_v):
    self.v.assign(new_v)

module = CustomModule()

Когда вы сохраняете `tf.Module` , `tf.Variable` атрибуты `tf.function` методы `tf.Module` найденные с помощью рекурсивного обхода. (Подробнее об этом рекурсивном обходе см. В [учебнике](./checkpoint.ipynb) по контрольным точкам.) Однако все атрибуты, функции и данные Python теряются. Это означает, что при `tf.function` код Python не сохраняется.

Если код Python не сохранен, откуда SavedModel знает, как восстановить функцию?

Вкратце, `tf.function` работает путем отслеживания кода Python для создания ConcreteFunction (вызываемой оболочки вокруг `tf.Graph` ). `tf.function` , вы действительно сохраняете `tf.function` для ConcreteFunctions.

Чтобы узнать больше о взаимосвязи между `tf.function` и ConcreteFunctions, см. [Руководство tf.function](../../guide/concrete_function) . В этом [разделе, посвященном трассировке,](../../tutorials/customization/performance#side_effects_in_tffunction) также уточняется, какой код Python захватывается ConcreteFunction.

In [0]:
module_no_signatures_path = os.path.join(tmpdir, 'module_no_signatures')
module(tf.constant(0.))
print('Saving model...')
tf.saved_model.save(module, module_no_signatures_path)

## Загрузка и использование нестандартной модели

Когда вы загружаете SavedModel в Python, все атрибуты `tf.Variable` `tf.function` методы и `tf.Module` восстанавливаются в той же структуре объекта, что и исходный сохраненный `tf.Module` .

In [0]:
imported = tf.saved_model.load(module_no_signatures_path)
assert imported(tf.constant(3.)).numpy() == 3
imported.mutate(tf.constant(2.))
assert imported(tf.constant(3.)).numpy() == 6

Поскольку код Python не сохраняется, вызов `tf.function` с новой входной сигнатурой завершится ошибкой:

```python
imported(tf.constant([3.]))
```

<pre>ValueError: Could not find matching function to call for canonicalized inputs ((&lt;tf.tensor shape="(1,)" dtype="float32"&gt;,), {}). Only existing signatures are [((TensorSpec(shape=(), dtype=tf.float32, name=u'x'),), {})].
&lt;/tf.tensor&gt;</pre>

### Базовая тонкая настройка

Доступны переменные объекты, и мы можем выполнять обратную передачу с помощью импортированных функций. Этого достаточно для точной настройки (т.е. переобучения) SavedModel в простых случаях.

In [0]:
optimizer = tf.optimizers.SGD(0.05)

def train_step():
  with tf.GradientTape() as tape:
    loss = (10. - imported(tf.constant(2.))) ** 2
  variables = tape.watched_variables()
  grads = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(grads, variables))
  return loss

In [0]:
for _ in range(10):
  # "v" approaches 5, "loss" approaches 0
  print("loss={:.2f} v={:.2f}".format(train_step(), imported.v.numpy()))

### Общая тонкая настройка

SavedModel от Keras предоставляет [больше деталей,](https://github.com/tensorflow/community/blob/master/rfcs/20190509-keras-saved-model.md#serialization-details) чем простой `__call__` для решения более сложных случаев тонкой настройки. TensorFlow Hub рекомендует предоставить следующие из них, если применимо, в общих моделях SavedModels с целью точной настройки:

- Если в модели используется выпадение или другой метод, в котором прямой проход отличается между обучением и логическим выводом (например, пакетная нормализация), `__call__` `training=` оцениваемый Python, который по умолчанию имеет значение `False` но может иметь значение `True` .
- Рядом с атрибутом `__call__` `.variable` и `.trainable_variable` с соответствующими списками переменных. Переменная, которая изначально была обучаемой, но должна быть заморожена во время точной настройки, опускается в `.trainable_variables` .
- Для таких фреймворков, как Keras, которые представляют регуляризаторы веса как атрибуты слоев или `.regularization_losses` , также может быть атрибут .regularization_losses. Он содержит список функций с нулевым аргументом, значения которых предназначены для добавления к общей потере.

Возвращаясь к первоначальному примеру MobileNet, мы можем увидеть некоторые из них в действии:

In [0]:
loaded = tf.saved_model.load(mobilenet_save_path)
print("MobileNet has {} trainable variables: {}, ...".format(
          len(loaded.trainable_variables),
          ", ".join([v.name for v in loaded.trainable_variables[:5]])))

In [0]:
trainable_variable_ids = {id(v) for v in loaded.trainable_variables}
non_trainable_variables = [v for v in loaded.variables
                           if id(v) not in trainable_variable_ids]
print("MobileNet also has {} non-trainable variables: {}, ...".format(
          len(non_trainable_variables),
          ", ".join([v.name for v in non_trainable_variables[:3]])))

## Указание подписей при экспорте

Такие инструменты, как TensorFlow Serving и `saved_model_cli` могут взаимодействовать с SavedModels. Чтобы помочь этим инструментам определить, какие конкретные функции использовать, нам нужно указать обслуживающие подписи. `tf.keras.Model` автоматически определяет обслуживающие подписи, но нам придется явно объявить обслуживающую подпись для наших настраиваемых модулей.

По умолчанию в настраиваемом `tf.Module` подписи не объявляются.

In [0]:
assert len(imported.signatures) == 0

Чтобы объявить обслуживающую сигнатуру, укажите ConcreteFunction, используя `signatures` kwarg. При указании одной подписи ключ подписи будет иметь значение `'serving_default'` , которое сохраняется как константа `tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY` .

In [0]:
module_with_signature_path = os.path.join(tmpdir, 'module_with_signature')
call = module.__call__.get_concrete_function(tf.TensorSpec(None, tf.float32))
tf.saved_model.save(module, module_with_signature_path, signatures=call)

In [0]:
imported_with_signatures = tf.saved_model.load(module_with_signature_path)
list(imported_with_signatures.signatures.keys())


Чтобы экспортировать несколько подписей, передайте словарь ключей подписей в ConcreteFunctions. Каждый ключ подписи соответствует одной ConcreteFunction.

In [0]:
module_multiple_signatures_path = os.path.join(tmpdir, 'module_with_multiple_signatures')
signatures = {"serving_default": call,
              "array_input": module.__call__.get_concrete_function(tf.TensorSpec([None], tf.float32))}

tf.saved_model.save(module, module_multiple_signatures_path, signatures=signatures)

In [0]:
imported_with_multiple_signatures = tf.saved_model.load(module_multiple_signatures_path)
list(imported_with_multiple_signatures.signatures.keys())

По умолчанию имена выходных тензоров довольно общие, например `output_0` . Чтобы управлять именами выходов, измените `tf.function` чтобы она возвращала словарь, который сопоставляет имена выходов с выходами. Имена входов являются производными от имен аргументов функции Python.

In [0]:
class CustomModuleWithOutputName(tf.Module):
  def __init__(self):
    super(CustomModuleWithOutputName, self).__init__()
    self.v = tf.Variable(1.)

  @tf.function(input_signature=[tf.TensorSpec([], tf.float32)])
  def __call__(self, x):
    return {'custom_output_name': x * self.v}

module_output = CustomModuleWithOutputName()
call_output = module_output.__call__.get_concrete_function(tf.TensorSpec(None, tf.float32))
module_output_path = os.path.join(tmpdir, 'module_with_output_name')
tf.saved_model.save(module_output, module_output_path,
                    signatures={'serving_default': call_output})

In [0]:
imported_with_output_name = tf.saved_model.load(module_output_path)
imported_with_output_name.signatures['serving_default'].structured_outputs

## Сохраненные модели из оценщиков

Оценщики экспортируют SavedModels через [`tf.Estimator.export_saved_model`](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator#export_saved_model) . Подробности см. В [руководстве по Оценщику](https://www.tensorflow.org/guide/estimator) .

In [0]:
input_column = tf.feature_column.numeric_column("x")
estimator = tf.estimator.LinearClassifier(feature_columns=[input_column])

def input_fn():
  return tf.data.Dataset.from_tensor_slices(
    ({"x": [1., 2., 3., 4.]}, [1, 1, 0, 0])).repeat(200).shuffle(64).batch(16)
estimator.train(input_fn)

serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec([input_column]))
estimator_base_path = os.path.join(tmpdir, 'from_estimator')
estimator_path = estimator.export_saved_model(estimator_base_path, serving_input_fn)

Эта SavedModel принимает сериализованные `tf.Example` протокола tf.Example, которые полезны для обслуживания. Но мы также можем загрузить его с помощью `tf.saved_model.load` и запустить из Python.

In [0]:
imported = tf.saved_model.load(estimator_path)

def predict(x):
  example = tf.train.Example()
  example.features.feature["x"].float_list.value.extend([x])
  return imported.signatures["predict"](
    examples=tf.constant([example.SerializeToString()]))

In [0]:
print(predict(1.5))
print(predict(3.5))

`tf.estimator.export.build_raw_serving_input_receiver_fn` позволяет вам создавать функции ввода, которые принимают необработанные тензоры, а не `tf.train.Example` s.

## Загрузить SavedModel в C ++

[Версия загрузчика](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/cc/saved_model/loader.h) SavedModel для C ++ предоставляет API для загрузки SavedModel из пути, при этом разрешая SessionOptions и RunOptions. Вы должны указать теги, связанные с загружаемым графиком. Загруженная версия SavedModel называется SavedModelBundle и содержит MetaGraphDef и сеанс, в котором он загружен.

```C++
const string export_dir = ...
SavedModelBundle bundle;
...
LoadSavedModel(session_options, run_options, export_dir, {kSavedModelTagTrain},
               &bundle);
```

<a id="saved_model_cli"></a>

## Подробная информация об интерфейсе командной строки SavedModel

Вы можете использовать интерфейс командной строки (CLI) SavedModel для проверки и выполнения SavedModel. Например, вы можете использовать CLI для проверки `SignatureDef` модели. Интерфейс командной строки позволяет быстро подтвердить, что входной тип и форма тензора соответствуют модели. Более того, если вы хотите протестировать свою модель, вы можете использовать CLI для проверки работоспособности, передав образцы входных данных в различных форматах (например, выражения Python) и затем извлекая выходные данные.

### Установите интерфейс командной строки SavedModel

Вообще говоря, вы можете установить TensorFlow одним из следующих двух способов:

- Установив предварительно созданный двоичный файл TensorFlow.
- Создав TensorFlow из исходного кода.

Если вы установили TensorFlow через предварительно созданный двоичный файл TensorFlow, тогда в вашей системе уже установлен интерфейс командной строки SavedModel по пути `bin/saved_model_cli` .

`saved_model_cli` из исходного кода, вы должны выполнить следующую дополнительную команду для сборки saved_model_cli:

```
$ bazel build tensorflow/python/tools:saved_model_cli
```

### Обзор команд

Интерфейс командной строки SavedModel поддерживает следующие две команды в SavedModel:

- `show` , в котором показаны вычисления, доступные из SavedModel.
- `run` , который запускает вычисление из SavedModel.

### `show` команду

SavedModel содержит один или несколько вариантов модели (технически `v1.MetaGraphDef` s), идентифицируемых их наборами тегов. Чтобы служить модели, вы можете задаться вопросом, какие `SignatureDef` есть в каждом варианте модели и каковы их входы и выходы. Команда `show` позволяет вам исследовать содержимое SavedModel в иерархическом порядке. Вот синтаксис:

```
usage: saved_model_cli show [-h] --dir DIR [--all]
[--tag_set TAG_SET] [--signature_def SIGNATURE_DEF_KEY]
```

Например, следующая команда показывает все доступные наборы тегов в SavedModel:

```
$ saved_model_cli show --dir /tmp/saved_model_dir
The given SavedModel contains the following tag-sets:
serve
serve, gpu
```

Следующая команда показывает все доступные `SignatureDef` для набора тегов:

```
$ saved_model_cli show --dir /tmp/saved_model_dir --tag_set serve
The given SavedModel `MetaGraphDef` contains `SignatureDefs` with the
following keys:
SignatureDef key: "classify_x2_to_y3"
SignatureDef key: "classify_x_to_y"
SignatureDef key: "regress_x2_to_y3"
SignatureDef key: "regress_x_to_y"
SignatureDef key: "regress_x_to_y2"
SignatureDef key: "serving_default"
```

Если *в наборе тегов несколько* тегов, необходимо указать все теги, каждый тег должен быть разделен запятой. Например:

<pre>$ saved_model_cli show --dir /tmp/saved_model_dir --tag_set serve,gpu
</pre>

Чтобы показать все входы и выходы TensorInfo для определенного `SignatureDef` , передайте ключ `SignatureDef` в параметр `signature_def` Это очень полезно, когда вы хотите узнать значение ключа тензора, dtype и форму входных тензоров для последующего выполнения графа вычислений. Например:

```
$ saved_model_cli show --dir \
/tmp/saved_model_dir --tag_set serve --signature_def serving_default
The given SavedModel SignatureDef contains the following input(s):
  inputs['x'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: x:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['y'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: y:0
Method name is: tensorflow/serving/predict
```

Чтобы показать всю доступную информацию в SavedModel, используйте параметр `--all` . Например:

<pre>$ saved_model_cli show --dir /tmp/saved_model_dir --all
MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['classify_x2_to_y3']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: x2:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: y3:0
  Method name is: tensorflow/serving/classify

...

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: x:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['y'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: y:0
  Method name is: tensorflow/serving/predict
</pre>

### `run` команду

Вызовите команду `run` чтобы запустить вычисление графика, передавая входные данные и затем отображая (и при необходимости сохраняя) выходные данные. Вот синтаксис:

```
usage: saved_model_cli run [-h] --dir DIR --tag_set TAG_SET --signature_def
                           SIGNATURE_DEF_KEY [--inputs INPUTS]
                           [--input_exprs INPUT_EXPRS]
                           [--input_examples INPUT_EXAMPLES] [--outdir OUTDIR]
                           [--overwrite] [--tf_debug]
```

Команда `run` предоставляет следующие три способа передачи входных данных в модель:

- `--inputs` позволяет передавать в файлах numpy ndarray.
- `--input_exprs` позволяет передавать выражения Python.
- `--input_examples` позволяет передавать `tf.train.Example` .

#### `--inputs`

Чтобы передать входные данные в файлы, укажите параметр `--inputs` , который принимает следующий общий формат:

```bsh
--inputs <INPUTS>
```

где *INPUTS* - один из следующих форматов:

- `<input_key>=<filename>`
- `<input_key>=<filename>[<variable_name>]`

Вы можете передать несколько *ВХОДОВ* . Если вы передаете несколько входов, используйте точку с запятой для разделения каждого из *ВХОДОВ* .

`saved_model_cli` использует `numpy.load` для загрузки имени *файла* . Имя *файла* может быть в любом из следующих форматов:

- `.npy`
- `.npz`
- формат рассола

`.npy` всегда содержит numpy ndarray. Следовательно, при загрузке из `.npy` содержимое будет напрямую назначено указанному входному тензору. Если вы укажете *имя_переменной* с этим файлом `.npy` *, имя_переменной* будет проигнорировано и будет выдано предупреждение.

При загрузке из `.npz` (zip) вы можете дополнительно указать *имя_переменной,* чтобы идентифицировать переменную в zip-файле для загрузки для входного тензорного ключа. Если вы не укажете *имя_переменной* , интерфейс командной строки SavedModel проверит, что в zip-файл включен только один файл, и загрузит его для указанного входного тензорного ключа.

При загрузке из файла pickle, если `variable_name` , все, что находится внутри файла pickle, будет передано указанному входному тензорному ключу. В противном случае интерфейс командной строки SavedModel будет предполагать, что словарь хранится в файле pickle, и будет использоваться значение, соответствующее *переменной_name* .

#### `--input_exprs`

Чтобы передавать входные данные через выражения Python, укажите параметр `--input_exprs` . Это может быть полезно, когда у вас нет файлов данных, но вы все еще хотите проверить работоспособность модели с помощью некоторых простых входных данных, которые соответствуют dtype и форме `SignatureDef` модели. Например:

```bsh
`<input_key>=[[1],[2],[3]]`
```

В дополнение к выражениям Python вы также можете передавать numpy-функции. Например:

```bsh
`<input_key>=np.ones((32,32,3))`
```

(Обратите внимание, что `numpy` уже доступен вам как `np` .)

#### `--input_examples`

Чтобы передать `tf.train.Example` качестве входных данных, укажите параметр `--input_examples` . Для каждого ключа ввода требуется список словарей, где каждый словарь является экземпляром `tf.train.Example` . Ключи словаря - это функции, а значения - это списки значений для каждой функции. Например:

```bsh
`<input_key>=[{"age":[22,24],"education":["BS","MS"]}]`
```

#### Сохранить вывод

По умолчанию интерфейс командной строки SavedModel записывает вывод в стандартный вывод. Если каталог передается `--outdir` , выходные данные будут сохранены как `.npy` названные в честь выходных тензорных ключей в данном каталоге.

Используйте `--overwrite` чтобы перезаписать существующие выходные файлы.
